In [ ]:
## Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
## Spatial Packages
import geopandas as gpd
from pylab import figure, scatter, show
import pysal as ps
from splot.libpysal import plot_spatial_weights
import libpysal
import mapclassify as mc
import esda 


Get some data. We have been using the: 
* food 
* cshapes

But now lets also use the region file, so we can look at regions if we want.

In [ ]:
def get_data():
    
    food = pd.read_csv('data/food_all.csv', encoding='latin-1')
    food_sum = food.groupby('ccode')['pt_attempt', 'pt_suc'].sum().reset_index()
    
    
    world = gpd.read_file('data/worldmap/cshapes.shp')
    world = world.loc[world.COWEYEAR == 2016]
    
    base = world.merge(food_sum, left_on='COWCODE', right_on='ccode', how='left')
    
    region = pd.read_csv('data/codeswregions.csv')
    
    base = base.merge(region, on='ccode', how='left')
    base['pt_attempt'].fillna(0, inplace=True)
    base['pt_suc'].fillna(0, inplace=True)
    return base

df = get_data()

Quick plot of pt_attempt which are coups

In [ ]:
fig, ax= plt.subplots(figsize=(15,10), dpi=120)
df.plot(column='pt_attempt', ax=ax, legend=True, legend_kwds={'orientation':
                                                              'horizontal'})
plt.title("Total Coups")
ax.set_aspect('equal')

Zoom in on SEASIA with ax.set_xlim and ax.set_ylim

In [ ]:
fig, ax= plt.subplots(figsize=(15,10), dpi=120)
df.plot(column='pt_attempt', ax=ax, legend=True)
plt.title("Total Coups")
ax.set_xlim(50,150)
ax.set_ylim(-20,40)
ax.set_aspect('equal')

Instead of a continuous variable for coups, lets bin it qith the scheme='Quantiles' feature

In [ ]:
fig, ax= plt.subplots(figsize=(15,10), dpi=120)
df.plot(column='pt_attempt',  edgecolor='black',scheme='Quantiles', k=5, cmap='GnBu',legend=True,ax=ax)
plt.title("Total Coups")
ax.set_aspect('equal')


Make the weights like in the neighborhoods exercise

In [ ]:
def basic_weights(shape, kernel_band = 20, kn = 5):
    
    queen = ps.lib.weights.contiguity.Queen.from_dataframe(shape)
    queen.transform = 'r'
    
    rook = ps.lib.weights.contiguity.Rook.from_dataframe(shape)
    rook.transform = 'r'
    
    kernel = ps.lib.weights.Kernel.from_dataframe(shape, bandwidth=kernel_band)
    kernel.transform = 'r'
    
    knn = ps.lib.weights.KNN.from_dataframe(shape, k=kn)
    knn.transform = 'r'
    
    return queen, rook, kernel, knn

queen, rook, kernel, knn = basic_weights(df)

Lets make a spatial lag!

In [ ]:
y = df['pt_attempt']

ylag = ps.lib.weights.lag_spatial(queen, y)

That's it, all you have to do is pass the lag_spatial both the variable you want to lag: y

and the neighborhood weight:
queen

We can make some quantiles now and plot

In [ ]:
ylagq5 = mc.Quantiles(ylag, k=5)

In [ ]:
df['ylag_q'] = ylag

In [ ]:
fig, ax= plt.subplots(figsize=(15,10), dpi=120)
df.assign(ylag_queen=ylagq5.yb).plot(column='ylag_queen', edgecolor='black',categorical=True, k=5, cmap='GnBu',
                                 legend=True,ax=ax)
plt.title("Total Coups")
ax.set_aspect('equal')

Lets plot with both coups, and the spatial lag so we can see them together.

In [ ]:
fig, (ax1,ax2)= plt.subplots(figsize=(24,15), dpi=160, nrows=2)

df.plot(column='ylag_q', edgecolor='black',scheme='Quantiles', k=5, cmap='GnBu',
                                 legend=True,ax=ax2)

df.plot(column='pt_attempt',  edgecolor='black',scheme='Quantiles', k=5, cmap='GnBu',
          legend=True,ax=ax1)

ax2.set_title("Coup Spatial Lag")
ax1.set_title("Total Coups")
ax1.set_aspect('equal')
ax2.set_aspect('equal')

# Moran's I

Again we just need the lag and the neighborhood

In [ ]:
np.random.seed(7)
mi = esda.moran.Moran(y, queen)


In [ ]:
print("Moran's I Coefficient: ")
mi.I

In [ ]:
print("Moran's I P Values: ")
mi.p_sim

# LISA

In [ ]:
coups = df['pt_attempt']
li = esda.moran.Moran_Local(coups, queen)

In [ ]:
sig = 1 * (li.p_sim < 0.05)
hotspot = 1 * (sig * li.q==1)
coldspot = 3 * (sig * li.q==3)
doughnut = 2 * (sig * li.q==2)
diamond = 4 * (sig * li.q==4)
spots = hotspot + coldspot + doughnut + diamond
spots

In [ ]:
sig = li.p_sim < 0.05
hotspot = sig * li.q==1
coldspot = sig * li.q==3
doughnut = sig * li.q==2
diamond = sig * li.q==4

In [ ]:
spot_labels = [ 'Not Significant', 'Hot Spot', 'Doughnut', 'Cold Spot', 'Diamond']
labels = [spot_labels[i] for i in spots]

In [ ]:
from matplotlib import colors
hmap = colors.ListedColormap(['#80b1d3','#bebada', '#8dd3c7', '#fb8072', '#ffffb3'])


fig, ax= plt.subplots(figsize=(15,10), dpi=120)
df.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='black', legend=True)
#ax.set_facecolor('black')
plt.title('Local Spatial Autocorrelation')
plt.show()

In [ ]:
fig, ax= plt.subplots(figsize=(15,10), dpi=150)
df.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='black', legend=True)
#ax.set_facecolor('black')
plt.title('Local Spatial Autocorrelation', fontsize=22, y=1.08)
ax.set_xlim(-90, -30)
ax.set_ylim(-60, 20)

plt.show()

In [ ]:
fig, ax= plt.subplots(figsize=(15,10), dpi=150)
df.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='black', legend=True)
#ax.set_facecolor('black')
plt.title('Local Spatial Autocorrelation', fontsize=22, y=1.08)
ax.set_xlim(-22, 50)
ax.set_ylim(0, 40)
plt.show()
